## Bewolf_cluster_for_WRF_EMS

1. The four domain running of WRF EMS with reoslution startng from 27km, 9km, 3km to 1km gives error of ```exit with status of -9```,            
   [this](http://stackoverflow.com/questions/18529452/sudden-exit-with-status-of-9) showing that it would because of outof memeory status([more from this])(http://forum.wrfforum.com/viewtopic.php?f=6&t=407), this gives motivation further for cluster running of wrf ems
2. passwordless SSH is not suffecieidnt for cluster running of WRF EMS, as specfied in run_ncpus.conf, the created cluster has to be checked with
   netcheck script given with WRF-EMS, it is in strc folder [hint](http://strc.comet.ucar.edu/library/lists/wrfems/2011-January/000808.html)
2. So while running netcheck with passwordless ssh, it gives error that ```ssh localhostname hostname``` is not doing pasword less ssh, in the      begning, this error seems to be ridiculus, how and what! ssh is needed for local computer!
3. After a while repeated error on this genre, it is decided to go with normal cluster creation and try with the netcheck script again.
4. In the meantime it is found that host name lookup is varying based on server and Desktop, ```nwlookup -A``` shows no repaly in Dekstop Ubuntu    and in server it returns hostname with its satus 
5. So googling further reveals that this discripancy might be due to hsot name decripancy or static ip difference.
6. to resolve the doubt of static ip difference, as follwoing ubuntu server edited the file ```sudo nano /etc/network/interfaces``` and added the ip address given in [this](http://www.howtoforge.com/linux-basics-set-a-static-ip-on-ubuntu), but ater doing this after a reboot the LAN is not showing in the network manager, then found that the utilities networkmanager and network interfaces are contradaataory complementaroies [ref](http://askubuntu.com/questions/7079/how-do-you-configure-desktop-for-a-static-ip-address)[ref2](http://askubuntu.com/questions/168670/connection-not-visible-or-not-available-after-switch-dsl-to-lan) so revert back to intitial. the relaity was static ip address was earlier setup useing network manager, so this problem doubt is leftout
7. For Host name decripancy, followed [this](http://sysblogd.wordpress.com/2008/05/16/ubuntu-changing-hostname-from-command-line/) and from this    switched to make simple bewolf cluster from ubuntu so followed [this](http://rashmi2592.wordpress.com/2014/02/06/building-a-beowulf-cluster-in-ubuntu-12-04-lts/) it satrts from host name editing of the file ```sudo edit /etc/hosts``` in hoopoe(master) and dhcppc3(node1). this tutorial was then replaced with more advanced and comphrehenive [tut](http://byobu.info/article/Building_a_simple_Beowulf_cluster_with_Ubuntu/). the steps invoves as follows, as heading given in [tut](http://byobu.info/article/Building_a_simple_Beowulf_cluster_with_Ubuntu/)

#### Add the nodes to the hosts file of master
  ```
  sudo nano /etc/hosts
  127.0.0.1	localhost
  192.168.1.79	master
  192.168.1.100	node1
  ```
- and to the node1 as 
  ```
  sudo nano /etc/hosts
  127.0.0.1	localhost
  192.168.1.100	node1
  192.168.1.79	master
  ```

#### test the ping

ping master or ping node1 from vice versa computer, it has to ping without error
  
#### Defining a user for running MPI jobs

  a new user was added in node1 so this was not followed, this user addition in the begning made due course hostname resolve step. While adding the user giving the identical uuid for master and node will avaoid nfs folder write error later. It can be done by ```sudo adduser username --uid uidofmaster```, then installing nfs common ```sudo apt-get install nfs-common``` tells user is not in sudoers group, for this logout to a user name having sudo access and can be done the isntallation. But latter to mount the nfs folder sudo has to supplay, to solve this [follow](http://tuxthink.blogspot.in/2012/05/user-is-not-in-sudoers-file-this.html) ```sudo nano /etc/sudoers\ //add this line/ username ALL = (ALL) ALL ```

#### Install and setup the Network File System

- install nfs-kernel-server
```
master:~$ sudo apt-get install nfs-kernel-server
```
- followed these steps for chmod the user for the home folder
```
master:~$ ls -l /home/ | grep mpiuser
drwxr-xr-x   7 mpiuser mpiuser  4096 May 11 15:47 mpiuser
master:~$ sudo chown mpiuser:mpiuser /path/to/shared/dir
```
- then edited the ```/etc/exports``` added the line ```/home/mpiuser *(rw,sync,no_subtree_check)```
- then restarted the nfs server by ```master:~$ sudo service nfs-kernel-server restart```
- In node1 checked the nfs server running by ```node1:~$ showmount -e master``` it gives output of ```/home/mpiuser *```
- after this as per tut opeed the firewall in master by ```master:~$ sudo ufw allow from 192.168.1.0/24```
- to mount the opened nfs folder in node1 run this command ```node1:~$ sudo mount master:/home/mpiuser /home/mpiuser```
- after this step without any error, ```ls -al``` shows all the folders and file in the node1 computer!
- from node1 has to maeke a mount with master by
```sudo mount master:/home/mpiuser /home/mpiuser```

#### Setup passwordless SSH for communication between nodes

- this major step made earlier before nfs server folder mounting seems aboslute by reaading this step in the tut!, follwoed those steps and it returend error of permission denied(publickey) follwoing steps were followed from [here](https://help.ubuntu.com/community/MpichCluster) aslo refered fr this step by getting error
```
ssh­-keygen ­-t rsa
cat id_pub.dsa >> authorized_keys
```
- but after this step, doing ```ssh hostname``` returned the error of permission denied(publickey), it is of time consuming error took attmepts of keygenration and deletion, nfs unmount hanging in server, which was resolved by pressing key ```ALT+Prntscrren+REISUB``` to revoke the hanged server from force [restart](http://askubuntu.com/questions/4408/what-should-i-do-when-ubuntu-freezes). It seems the nfs folder mount and disruption of nfs folder can serieously hand ubuntu server, it has to solved by first stoping the nfs-kernel-server by ```sudo service nfs-kernel-server stop``` in master node, then umount the foldeer by going into root in node1 by ```sudo bash\ umount -f -l /home/hoopoe/```, based on [this](http://unix.stackexchange.com/questions/15024/umount-device-is-busy-why)
- to ssh permision error, after a long google search and multiple compination of key genraton!, it is settleed that the permission of .ssh folders makes difference, it is from obervation that, some time the node1 can't get into .ssh folder saying permission denied and also from [this](http://serverfault.com/questions/39733/why-do-i-get-permission-denied-publickey-when-trying-to-ssh-from-local-ubunt) and [this](http://stackoverflow.com/questions/7086678/permission-denied-publickey-when-connecting-to-aws-sever), installed ```sudo apt-get install policycoreutils``` to do this ```restorecon -r /home/*/.ssh``` as measure to rsolve the error, after this made changes of file foldeer permission as [follows](http://www.howtogeek.com/168119/fixing-warning-unprotected-private-key-file-on-linux/), this step was atained by running the ssh in this manner ```ssh -vvv oneadmin@YOU_HOST``` which gives error of ``` WARNING: UNPROTECTED PRIVATE KEY FILE!```
- so follwoed these steps 
```
sudo chmod 600 ~/.ssh/id_rsa
sudo chmod 600 ~/.ssh/id_rsa.pub
sudo chmod 644 ~/.ssh/known_hosts
sudo chmod 755 ~/.ssh
```
the ssh config files has to be edited ```sudo nano /etc/ssh/sshd_config``` and ssh has to be restarted ```sudo service ssh restart```
- after this steps the ssh can be accessed in node1 nfs folder, and made check of similar permison in master and node1 by runnin ```ls -alZ``` which give detailed perimion and uid
- by this step a further netcheckup was made and the ssh error goes away!, now the write permission erro due to the different UID between master and node1 username
- uid of user name was changed by followeing [this](http://askubuntu.com/questions/16700/how-can-i-change-my-own-user-id)
```
sudo passwd root
groupmod --gid NEWGID username
usermod  --uid NEWUID username 
passwd -dl root
```
- now again checking for netcheckup master node1, this time all went fine, now going for running wrf ems in cluster, hopefull of resolving exit code9!